In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
import os
import json
import shutil

## Multitask

In [3]:
experiment_name = 'multitask_resnet_fpn_transformer_decoder_ce'

first_stage_results_source_folder = "../data/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/ann_dir"
first_stage_results_destination_folder = f"../resnet_model/work_dirs/{experiment_name}/combine_first_and_second_stage_results"
second_stage_results_file = f"../resnet_model/work_dirs/{experiment_name}/results.json"

In [4]:
shutil.copytree(first_stage_results_source_folder, first_stage_results_destination_folder)

'../resnet_model/work_dirs/singletask_resnet_fpn_verb_and_target_parallel_fc_predict_on_train/combine_first_and_second_stage_results'

In [5]:
from utils.general.dataset_variables import TripletSegmentationVariables

VERB_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['verb']
TARGET_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['target']

In [6]:
# Load the second-stage results
def combine_multitask_output_second_stage_with_first_stage_result(first_stage_results_destination_folder,
                                                                  second_stage_results_file):

    with open(second_stage_results_file, 'r') as f:
        second_stage_results = json.load(f)

    # Iterate through all JSON files in the first-stage folder
    for file_name in os.listdir(first_stage_results_destination_folder):
        file_name_no_ext = file_name.split('.')[0]
        if file_name.endswith(".json"):
            first_stage_path = os.path.join(first_stage_results_destination_folder, file_name)
            
            # Load the first-stage JSON
            with open(first_stage_path, 'r') as f:
                first_stage_data = json.load(f)

            # Iterate through each shape in the first-stage JSON
            for shape in first_stage_data['shapes']:
                # Construct the unique key used in the second-stage results
                
                image_key = f"{file_name_no_ext},{shape['label']},{shape['group_id']},{shape['verb']},{shape['target']}"
                
                # If the key exists in the second-stage results, update the shape
                if image_key in second_stage_results:
                    print(f'found: imagekey {image_key}')  
                    second_stage_info = second_stage_results[image_key]
                    shape['verb'] =  VERB_ID_TO_CLASS_DICT[str(second_stage_info['verb'] + 1)]
                    shape['target'] = TARGET_ID_TO_CLASS_DICT[str(second_stage_info['target'] + 1)]
                    shape['verb_score'] = second_stage_info.get('verb_score', None)
                    shape['target_score'] = second_stage_info.get('target_score', None)
                else: 
                    print(f'imagekey does not exist {image_key}')    

            # Save the updated JSON back to the folder
            with open(first_stage_path, 'w') as f:
                json.dump(first_stage_data, f, indent=4)

    print("Updated all first-stage JSONs with second-stage results.")

In [ ]:
combine_multitask_output_second_stage_with_first_stage_result(first_stage_results_destination_folder = first_stage_results_destination_folder,
                                                                  second_stage_results_file = second_stage_results_file )

## Singletask Verb and Singletask target

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
import os
import json
import shutil

In [ ]:
# val best
# first_stage_results_source_folder = "../data/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/ann_dir"
# first_stage_results_destination_folder = "../resnet_model/work_dirs/singletask_resnet_fpn_for_verb_and_target_best/combine_first_and_second_stage_results"
# second_stage_verb_results_file = "../resnet_model/work_dirs/singletask_resnet_fpn_for_verb_and_target_best/best_results_verb.json"
# second_stage_target_results_file = "../resnet_model/work_dirs/singletask_resnet_fpn_for_verb_and_target_best/best_results_target.json"

#debugging
# experiment_name = 'singletask_resnet_fpn_verb_and_target_parallel_fc_predict_on_train'
# first_stage_results_source_folder = "../data/triplet_segmentation_dataset_v2_second_stage/train/ann_dir"
# first_stage_results_destination_folder = f"../resnet_model/work_dirs/{experiment_name}/combine_first_and_second_stage_results"
# second_stage_verb_results_file = f"../resnet_model/work_dirs/{experiment_name}/results_verb.json"
# second_stage_target_results_file = f"../resnet_model/work_dirs/{experiment_name}/results_target.json"

# latest 
experiment_name = 'singletask_resnet_fpn_verb_and_target_parallel_decoders'
first_stage_results_source_folder = "../data/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/ann_dir"
first_stage_results_destination_folder = f"../resnet_model/work_dirs/{experiment_name}/combine_first_and_second_stage_results"
second_stage_verb_results_file = f"../resnet_model/work_dirs/{experiment_name}/results_verb.json"
second_stage_target_results_file = f"../resnet_model/work_dirs/{experiment_name}/results_target.json"

In [7]:
shutil.copytree(first_stage_results_source_folder, first_stage_results_destination_folder)

'../resnet_model/work_dirs/threetask_resnet_fpn_parallel_decoders/combine_first_and_second_stage_results'

In [9]:
from utils.general.dataset_variables import TripletSegmentationVariables

VERB_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['verb']
TARGET_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['target']

In [10]:
# Load the second-stage results
def combine_singletask_outputs_second_stage_with_first_stage_result(first_stage_results_destination_folder,
                                                                  second_stage_verb_results_file, 
                                                                  second_stage_target_results_file
                                                                  ):

    with open(second_stage_verb_results_file, 'r') as f:
        second_stage_verb_results = json.load(f)
        
    with open(second_stage_target_results_file, 'r') as f:
        second_stage_target_results = json.load(f)     

    # Iterate through all JSON files in the first-stage folder
    for file_name in os.listdir(first_stage_results_destination_folder):
        file_name_no_ext = file_name.split('.')[0]
        if file_name.endswith(".json"):
            first_stage_path = os.path.join(first_stage_results_destination_folder, file_name)
            
            # Load the first-stage JSON
            with open(first_stage_path, 'r') as f:
                first_stage_data = json.load(f)

            # Iterate through each shape in the first-stage JSON
            for shape in first_stage_data['shapes']:
                # Construct the unique key used in the second-stage results
                
                image_key = f"{file_name_no_ext},{shape['label']},{shape['group_id']},None,None"
                # image_key = f"{file_name_no_ext},{shape['label']},{shape['group_id']},{shape['verb']},{shape['target']}"
                
                # If the key exists in the second-stage results, update the shape
                if (image_key in second_stage_verb_results) and (image_key in second_stage_target_results):
                    print(f'found:imagekey {image_key}')  
                    second_stage_verb_info = second_stage_verb_results[image_key]
                    second_stage_target_info = second_stage_target_results[image_key]
                    shape['verb'] =  VERB_ID_TO_CLASS_DICT[str(second_stage_verb_info['verb'] + 1)]
                    shape['target'] = TARGET_ID_TO_CLASS_DICT[str(second_stage_target_info['target'] + 1)]
                    shape['verb_score'] = second_stage_verb_info.get('verb_score', None)
                    shape['target_score'] = second_stage_target_info.get('target_score', None)
                else: 
                    print(f'imagekey does not exist {image_key}')    

            # Save the updated JSON back to the folder
            with open(first_stage_path, 'w') as f:
                json.dump(first_stage_data, f, indent=4)

    print("Updated all first-stage JSONs with second-stage single stage verb results and target results.")

In [11]:
combine_singletask_outputs_second_stage_with_first_stage_result(first_stage_results_destination_folder = first_stage_results_destination_folder,
                                                                  second_stage_verb_results_file = second_stage_verb_results_file, 
                                                                  second_stage_target_results_file = second_stage_target_results_file )

FileNotFoundError: [Errno 2] No such file or directory: '../resnet_model/work_dirs/threetask_resnet_fpn_parallel_decoders/results_verb.json'

## Singletask VerbTarget Combined

In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import json
import shutil

In [7]:
# val best
# first_stage_results_source_folder = "../data/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/ann_dir"
# first_stage_results_destination_folder = "../resnet_model/work_dirs/singletask_resnet_fpn_for_verbtarget_best/combine_first_and_second_stage_results"
# second_stage_verbtarget_results_file = "../resnet_model/work_dirs/singletask_resnet_fpn_for_verbtarget_best/results_best.json"

# latest 
# experiment = 'threetask_resnet_fpn_parallel_decoders'
# first_stage_results_source_folder = "../data/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/ann_dir"
# first_stage_results_destination_folder = f"../resnet_model/work_dirs/{experiment}/combine_first_and_second_stage_results"
# second_stage_verbtarget_results_file = f"../resnet_model/work_dirs/{experiment}/results.json"

experiment = 'threetask_resnet_fpn_parallel_decoders'
first_stage_results_source_folder = "../../datasets/my_triplet_seg_datasets/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/ann_dir"
first_stage_results_destination_folder = f"../resnet_model/work_dirs/{experiment}/combine_first_and_second_stage_results"
second_stage_verbtarget_results_file = f"../resnet_model/work_dirs/{experiment}/results.json"

In [8]:
shutil.copytree(first_stage_results_source_folder, first_stage_results_destination_folder)

'../resnet_model/work_dirs/threetask_resnet_fpn_parallel_decoders/combine_first_and_second_stage_results'

In [9]:
from utils.general.dataset_variables import TripletSegmentationVariables

VERBTARGET_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['verbtarget']

In [10]:
# Load the second-stage results
def combine_singletask_verbtarget_outputs_second_stage_with_first_stage_result(first_stage_results_destination_folder,
                                                                  second_stage_verbtarget_results_file,
                                                                  ):

    with open(second_stage_verbtarget_results_file, 'r') as f:
        second_stage_verbtarget_results_file = json.load(f)
          

    # Iterate through all JSON files in the first-stage folder
    for file_name in os.listdir(first_stage_results_destination_folder):
        file_name_no_ext = file_name.split('.')[0]
        if file_name.endswith(".json"):
            first_stage_path = os.path.join(first_stage_results_destination_folder, file_name)
            
            # Load the first-stage JSON
            with open(first_stage_path, 'r') as f:
                first_stage_data = json.load(f)

            # Iterate through each shape in the first-stage JSON
            for shape in first_stage_data['shapes']:
                # Construct the unique key used in the second-stage results
                
                image_key = f"{file_name_no_ext},{shape['label']},{shape['group_id']},None,None"
                
                # If the key exists in the second-stage results, update the shape
                if (image_key in second_stage_verbtarget_results_file):
                    print(f'found: imagekey {image_key}')  
                    second_stage_verbtarget_info = second_stage_verbtarget_results_file[image_key]
                    verb_target_name = VERBTARGET_ID_TO_CLASS_DICT[str(second_stage_verbtarget_info['verbtarget'] + 1)]
                    verb_name, target_name = verb_target_name.split(',')
                    
                    shape['verb'] =  verb_name
                    shape['target'] = target_name
                    shape['verb_score'] = second_stage_verbtarget_info.get('verb_score', None)
                    shape['target_score'] = second_stage_verbtarget_info.get('target_score', None)
                else: 
                    print(f'imagekey does not exist {image_key}')    

            # Save the updated JSON back to the folder
            with open(first_stage_path, 'w') as f:
                json.dump(first_stage_data, f, indent=4)

    print("Updated all first-stage JSONs with second-stage single stage verb results and target results.")

In [11]:
combine_singletask_verbtarget_outputs_second_stage_with_first_stage_result(first_stage_results_destination_folder = first_stage_results_destination_folder,
                                                                  second_stage_verbtarget_results_file = second_stage_verbtarget_results_file  )

found: imagekey t50_VID14_000012,grasper,1,None,None
found: imagekey t50_VID14_000013,grasper,1,None,None
found: imagekey t50_VID14_000014,grasper,1,None,None
found: imagekey t50_VID14_000017,grasper,1,None,None
found: imagekey t50_VID14_000018,grasper,1,None,None
found: imagekey t50_VID14_000019,grasper,1,None,None
found: imagekey t50_VID14_000020,grasper,1,None,None
found: imagekey t50_VID14_000021,grasper,1,None,None
found: imagekey t50_VID14_000022,grasper,1,None,None
found: imagekey t50_VID14_000023,grasper,1,None,None
found: imagekey t50_VID14_000024,grasper,1,None,None
found: imagekey t50_VID14_000025,grasper,1,None,None
found: imagekey t50_VID14_000026,grasper,1,None,None
found: imagekey t50_VID14_000119,hook,1,None,None
found: imagekey t50_VID14_000119,grasper,1,None,None
found: imagekey t50_VID14_000120,hook,1,None,None
found: imagekey t50_VID14_000120,grasper,1,None,None
found: imagekey t50_VID14_000121,hook,1,None,None
found: imagekey t50_VID14_000121,grasper,1,None,None
fo